In [16]:
'''
This notebook is used to train the face-recognition model to learn a few users and then detect the user in a live feed
'''

import cv2
import face_recognition
import os

In [17]:
dataset = dict()
for imgFolder in os.listdir('data/'):
    dataset[imgFolder] = []
    for filename in os.listdir('data/' + imgFolder):
        filename = 'data/' + imgFolder + '/' + filename
        img = cv2.imread(filename)
        dataset[imgFolder].append(img)

len(dataset['6687'])

5

In [18]:
faces = dict()
for id in dataset:
    faces[id] = []
    for image in dataset[id]:
        face_locations = face_recognition.face_locations(image)
        for top, right, bottom, left in face_locations:
            face_image = image[top:bottom, left:right]
            faces[id].append(face_image)

len(faces['6687'])

4

In [19]:
known_faces = []
known_names = []

for id in faces:
    for face in faces[id]:
        face_encoding = face_recognition.face_encodings(face)
        if face_encoding != []:
            known_faces.append(face_encoding[0])
            known_names.append(id)

print(known_names[:25])
known_faces[:25]

['6687', '6687', '6687', '6687', '6779', '6779', '6779']


[array([-0.0883268 ,  0.07588337,  0.07129689, -0.01471252,  0.00756682,
        -0.04950783, -0.08281494, -0.04491476,  0.13132115, -0.1124001 ,
         0.24476406,  0.01551023, -0.16418681, -0.137348  ,  0.01755349,
         0.06665999, -0.23682186, -0.04101021, -0.00539387, -0.00622025,
         0.1091616 ,  0.0337486 ,  0.01576954,  0.07808951, -0.16836482,
        -0.32862151, -0.06332938, -0.12221634, -0.00685125, -0.16004919,
        -0.06085873,  0.03706448, -0.1812744 , -0.01489666, -0.06668081,
         0.05658157,  0.03830095, -0.03809707,  0.16904102,  0.03528561,
        -0.13013782,  0.00983342, -0.02851877,  0.2918773 ,  0.2330301 ,
        -0.01947954, -0.0020645 , -0.03266615,  0.1505072 , -0.25701043,
         0.03359739,  0.14746314,  0.04231233,  0.04674308,  0.07125086,
        -0.12440909, -0.01394989,  0.01455626, -0.09142385,  0.07104442,
         0.00402815, -0.0823456 ,  0.03371698,  0.01818366,  0.13768789,
         0.00732233, -0.09023052, -0.09848265,  0.0

In [20]:
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Detect faces in the frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Loop through each face in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for any of the known faces
        matches = face_recognition.compare_faces(known_faces, face_encoding)

        # If there is a match, label the face with the name
        name = "Unknown"
        if True in matches:
            match_index = matches.index(True)
            name = known_names[match_index]

        # Draw a box around the face and label it
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Quit the program if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
video_capture.release()
cv2.destroyAllWindows()